In [1]:
import pandas as pd

In [2]:
# we make the results in the bash shell using the saved output of the nanopore machine
# for f in *txt; do tail -n +6 "$f" >> results.txt; done
# need to read docs kenny sent and r/v

cols = ['Sample ID', 'User ID', 'Date', 'Time', 'ng/ul', 'A260', 'A280',
       '260/280', '260/230', 'Constant', 'Cursor Pos.', 'Cursor abs.',
       '340 raw', 'Measurement Type', 'Serial #', 'Config.', '220', '221',
       '222', '223', '224', '225', '226', '227', '228', '229', '230',
       '231', '232', '233', '234', '235', '236', '237', '238', '239',
       '240', '241', '242', '243', '244', '245', '246', '247', '248',
       '249', '250', '251', '252', '253', '254', '255', '256', '257',
       '258', '259', '260', '261', '262', '263', '264', '265', '266',
       '267', '268', '269', '270', '271', '272', '273', '274', '275',
       '276', '277', '278', '279', '280', '281', '282', '283', '284',
       '285', '286', '287', '288', '289', '290', '291', '292', '293',
       '294', '295', '296', '297', '298', '299', '300', '301', '302',
       '303', '304', '305', '306', '307', '308', '309', '310', '311',
       '312', '313', '314', '315', '316', '317', '318', '319', '320',
       '321', '322', '323', '324', '325', '326', '327', '328', '329',
       '330', '331', '332', '333', '334', '335', '336', '337', '338',
       '339', '340', '341', '342', '343', '344', '345', '346', '347',
       '348', '349', '350']

df = pd.read_table('results.txt', header=None)

In [3]:
df.columns = cols

In [4]:
df['Genomic DNA'] = df['ng/ul'].astype(float) * 3 # because we did a 1 in 2 dilution earlier

In [5]:
constants = {'final reaction vol':10,
             'DNA concn ng/ul in one Taqman well':10,
             'working volume ul':50,
             'working concentration ng/ul':100}

constants['dilution factor for working DNA (ng/ul) in Taqman well'] = \
constants['working concentration ng/ul'] / constants['DNA concn ng/ul in one Taqman well']

constants['volume of working DNA in Taqman well at (ng/ul)'] = \
constants['final reaction vol'] / constants['dilution factor for working DNA (ng/ul) in Taqman well']

In [6]:
df['diln factor for intermediate concn (ng/ul)'] = df['Genomic DNA'] / constants['working concentration ng/ul']
df['volume of genomic DNA for working concn (ng/ul)'] = constants['working volume ul'] / df['diln factor for intermediate concn (ng/ul)']
df['volume of PCR water for working concn (ng/ul)'] = constants['working volume ul'] - df['volume of genomic DNA for working concn (ng/ul)']

In [7]:
df.head()

,Sample ID,User ID,Date,Time,ng/ul,A260,A280,260/280,260/230,Constant,...,345,346,347,348,349,350,Genomic DNA,diln factor for intermediate concn (ng/ul),volume of genomic DNA for working concn (ng/ul),volume of PCR water for working concn (ng/ul)
0,NaN,Default,8/1/2018,12:16 PM,0,0,0,NaN,NaN,50,...,0,0,0,0,0,0,0.00,0.0000,inf,-inf
1,100005-1,Default,8/1/2018,12:16 PM,225.45,4.509,2.492,1.81,1.78,50,...,-0.003,-0.008,-0.008,-0.009,-0.01,-0.009,676.35,6.7635,7.392622,42.607378
2,140001,Default,8/1/2018,12:17 PM,10.37,0.207,0.167,1.24,1.19,50,...,0.009,0.009,0.009,0.006,0.006,0.006,31.11,0.3111,160.720026,-110.720026
3,140002,Default,8/1/2018,12:18 PM,15.34,0.307,0.222,1.38,0.96,50,...,0.005,0.004,0.008,0.008,0.006,0.008,46.02,0.4602,108.648414,-58.648414
4,140003,Default,8/1/2018,12:19 PM,48.77,0.975,0.551,1.77,1.81,50,...,0.002,0.005,0.008,0.008,0.007,0.01,146.31,1.4631,34.174014,15.825986


In [8]:
df['Date'] = df['Date'].map(lambda x: pd.to_datetime(x))

In [9]:
df = df.sort_values(by=['Date', 'Time'])

In [10]:
df.to_csv('results.csv')

In [11]:
# df = df.drop_duplicates(subset=["Sample ID"], keep='last') # keep most recent duplicate

In [12]:
df[['Date', 'Time', 'Sample ID', 'Genomic DNA', 'diln factor for intermediate concn (ng/ul)',
       'volume of genomic DNA for working concn (ng/ul)',
       'volume of PCR water for working concn (ng/ul)']].to_excel('carldata_kenny.xls', index=False)

In [13]:
constants

{'DNA concn ng/ul in one Taqman well': 10,
 'dilution factor for working DNA (ng/ul) in Taqman well': 10.0,
 'final reaction vol': 10,
 'volume of working DNA in Taqman well at (ng/ul)': 1.0,
 'working concentration ng/ul': 100,
 'working volume ul': 50}

In [14]:
df['260/280'] = df['260/280'].astype(float)

In [15]:
df['260/230'] = df['260/230'].astype(float)

In [16]:
df['ng/ul'] = df['ng/ul'].astype(float)

In [17]:
pd.to_datetime('2018')

Timestamp('2018-01-01 00:00:00')

In [18]:
df[df['Date'].map(lambda x: pd.to_datetime(x))  > pd.to_datetime('2019')][['Sample ID', 'Genomic DNA', '260/280', '230', '260/230','320']].describe()
# Purity of nucleic acid : 260/280 = 1.7-2  DNA/RNA -> 260 , protein (Aromatic aa ) -> 280
# High 230 => organics or chaotropic salts
# Carryover salt in sample: 260/230 >1.5 ; commonly 2.0 – 2.2
# The lower 260/230 ratio, the greater the salt present ( estimate of salt carryover in purified DNA )
# 260/230 >1.5-1.7 is best but doesn’t necessarily means DNA/RNA is unusable
# 320 => other contaminants are high 

,Genomic DNA,260/280,260/230
count,587.000000,561.000000,560.000000
mean,439.861073,1.730428,1.337679
std,577.865742,2.207462,0.648115
min,-232.650000,-42.590000,-1.930000
25%,35.535000,1.730000,0.770000
50%,254.820000,1.830000,1.440000
75%,663.330000,1.890000,1.840000
max,6397.710000,15.500000,3.670000


In [19]:
df[df['ng/ul'] > 100][['Sample ID', 'Genomic DNA', '260/280', '230', '260/230','320']].describe()
# Purity of nucleic acid : 260/280 = 1.7-2  DNA/RNA -> 260 , protein (Aromatic aa ) -> 280
# High 230 => organics or chaotropic salts
# Carryover salt in sample: 260/230 >1.5 ; commonly 2.0 – 2.2
# The lower 260/230 ratio, the greater the salt present ( estimate of salt carryover in purified DNA )
# 260/230 >1.5-1.7 is best but doesn’t necessarily means DNA/RNA is unusable
# 320 => other contaminants are high 

,Genomic DNA,260/280,260/230
count,548.000000,548.000000,547.000000
mean,1533.580073,1.802354,1.671846
std,2845.899299,0.230359,0.451632
min,300.540000,-2.860000,0.190000
25%,438.967500,1.770000,1.415000
50%,702.270000,1.820000,1.710000
75%,1149.892500,1.860000,1.980000
max,17516.160000,2.140000,3.670000


In [20]:
df[(df['260/280'].astype(float) > 1.7) & (df['260/280'].astype(float) < 2.0)] 
# Purity of nucleic acid : 260/280 = 1.7-2  DNA/RNA -> 260 , protein (Aromatic aa ) -> 280

,Sample ID,User ID,Date,Time,ng/ul,A260,A280,260/280,260/230,Constant,...,345,346,347,348,349,350,Genomic DNA,diln factor for intermediate concn (ng/ul),volume of genomic DNA for working concn (ng/ul),volume of PCR water for working concn (ng/ul)
54,080003-1,Default,2017-01-11,10:01,534.34,10.687,5.898,1.81,1.98,50,...,NaN,NaN,NaN,NaN,NaN,NaN,1603.02,16.0302,3.119113,46.880887
55,080003-1,Default,2017-01-11,10:03,428.95,8.579,4.688,1.83,1.92,50,...,NaN,NaN,NaN,NaN,NaN,NaN,1286.85,12.8685,3.885457,46.114543
56,080003-1,Default,2017-01-11,10:04,514.83,10.297,5.595,1.84,1.99,50,...,NaN,NaN,NaN,NaN,NaN,NaN,1544.49,15.4449,3.237315,46.762685
57,080003-2,Default,2017-01-11,10:06,147.40,2.948,1.632,1.81,1.40,50,...,NaN,NaN,NaN,NaN,NaN,NaN,442.20,4.4220,11.307101,38.692899
59,080004-1,Default,2017-01-11,10:09,52.78,1.056,0.603,1.75,0.99,50,...,NaN,NaN,NaN,NaN,NaN,NaN,158.34,1.5834,31.577618,18.422382
60,080004-2,Default,2017-01-11,10:11,1058.62,21.172,11.419,1.85,2.07,50,...,NaN,NaN,NaN,NaN,NaN,NaN,3175.86,31.7586,1.574377,48.425623
61,080005-1,Default,2017-01-11,10:12,159.46,3.189,1.712,1.86,1.65,50,...,NaN,NaN,NaN,NaN,NaN,NaN,478.38,4.7838,10.451942,39.548058
62,080005-2,Default,2017-01-11,10:14,114.36,2.287,1.232,1.86,1.51,50,...,NaN,NaN,NaN,NaN,NaN,NaN,343.08,3.4308,14.573860,35.426140
63,080006-1,Default,2017-01-11,10:15,146.97,2.939,1.537,1.91,1.86,50,...,NaN,NaN,NaN,NaN,NaN,NaN,440.91,4.4091,11.340183,38.659817
64,080006-2,Default,2017-01-11,10:16,211.51,4.23,2.261,1.87,1.95,50,...,NaN,NaN,NaN,NaN,NaN,NaN,634.53,6.3453,7.879848,42.120152


In [21]:
df['230'] # High 230 => organics or chaotropic salts

54        NaN
55        NaN
56        NaN
57        NaN
58        NaN
59        NaN
60        NaN
61        NaN
62        NaN
63        NaN
64        NaN
50      1.518
1361    1.518
51      0.954
1362    0.954
52      0.591
1363    0.591
53      5.373
1364    5.373
65          0
66      7.346
67      0.652
68      0.829
69      3.233
127         0
128     0.286
129     0.779
130     8.325
131     0.858
132     0.164
        ...  
874     0.045
875     0.307
876     0.357
877     0.228
878      0.06
879     2.656
880     0.444
881     0.057
882     0.154
883      0.62
884     4.712
885     0.507
886     0.581
1169        0
1170    0.239
1171    1.108
1172    2.582
1173     2.88
1174      0.4
1175     2.32
1176    0.682
1177    0.517
1178    1.381
1179    2.765
1180    2.748
1181    2.511
1182    0.794
1183    4.436
1184    2.054
1185    2.066
Name: 230, Length: 1417, dtype: object

In [22]:
df['260/230']
# Carryover salt in sample: 260/230 >1.5 ; commonly 2.0 – 2.2
# The lower 260/230 ratio, the greater the salt present ( estimate of salt carryover in purified DNA )
# 260/230 >1.5-1.7 is best but doesn’t necessarily means DNA/RNA is unusable

54      1.98
55      1.92
56      1.99
57      1.40
58      1.22
59      0.99
60      2.07
61      1.65
62      1.51
63      1.86
64      1.95
50      1.72
1361    1.72
51      1.59
1362    1.59
52      3.06
1363    3.06
53      2.09
1364    2.09
65       NaN
66      1.66
67      1.45
68      2.06
69      2.05
127      NaN
128     0.42
129     0.92
130     1.38
131     1.39
132     0.29
        ... 
874    -0.19
875     0.18
876     0.54
877     0.39
878     0.64
879     1.59
880     0.70
881     0.19
882     0.39
883     0.42
884     1.99
885     0.37
886     0.44
1169     NaN
1170    0.75
1171    1.07
1172    1.86
1173    2.20
1174    2.09
1175    2.15
1176    1.61
1177    1.56
1178    1.03
1179    2.16
1180    1.33
1181    1.83
1182    1.42
1183    1.45
1184    1.85
1185    1.27
Name: 260/230, Length: 1417, dtype: float64